In [1]:
"""This is video reader with all operations inside fnctions, so they can be called on 
    each frame out of a set of frames, and each video out of a set of videos
    """

'This is video reader with all operations inside fnctions, so they can be called on \n    each frame out of a set of frames, and each video out of a set of videos\n    '

In [2]:
"""Imports"""
import matplotlib.pyplot as plt #for maknig plots inside the notebook
import skimage
import skvideo.io
from skvideo.io import vreader, ffprobe
from skimage import measure, morphology
from skimage.filters import *
from operator import attrgetter
import numpy as np
import math
from itertools import chain
#from functools import partial, update_wrapper
from scipy import ndimage 
from pims import pipeline, Video
from glob import glob
import seaborn as sns
from collections import OrderedDict 
#import antigravity

In [3]:
"""Functions"""
# apply total thresholding to each of a list of frames; filter slides with wrong number
def total_threshold_filter(frame, frame_no, transition_threshold, class_thresh,
                           broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                           sides, classes, transitions): 
    
    m_lab = measure.label
    m_rop = measure.regionprops
    # returns a new list, different from the orginal
    thresh_img = frame > threshold_li(frame) # binary image
    img_labelled = m_lab(thresh_img) # contains connected regions
    properties_list = m_rop(img_labelled, coordinates = 'rc') # data about regions, for each connected region

    #----------getting maximum connected region----------
    biggest_r = max(properties_list, key = attrgetter('area'))

    biggest_r = m_rop(morphology.closing(m_lab(biggest_r.filled_image)), coordinates='rc')[0]
    #----------------------filter------------------------- can use actual filter
    test = (biggest_r.filled_area > area_threshold
               
               #and biggest_r.major_axis_length < major_threshold # must then be well-formed
               ) # major_axis length (< filterd) # particles must be in parallelogram
    if test: # keep frames that have enough paricles, and are not transitions
        if (biggest_r.inertia_tensor[0, 0] + biggest_r.inertia_tensor[1,1] > iner_threshold[0]
                and biggest_r.inertia_tensor[0, 0] + biggest_r.inertia_tensor[1,1] < iner_threshold[1]):
            i_threshed = frame > threshold_isodata(frame)
            img2_labelled = m_lab(i_threshed)
            properties2_list = m_rop(img2_labelled, coordinates = 'rc')
            biggest_r2 = max(properties2_list, key = attrgetter('area'))
            #-----------------------------classify pt 1---------------------------------------
            coprod = coord_prod(thresh_img, i_threshed)
            side = sideify(biggest_r2)
            clas = side
            sides[side].append(frame_no)
            classes[clas].append(frame_no)
            #print(clas)
        else:
            clas = 'ucf'
            
        if clas == 'ucf':
            #broken_count += 1
            #class_num = 0
            #print('ucf->reset class')
            pass
        else:
            if origin == 'ucf':
                origin = last_class
                #print("reset origin")
            if clas == last_class:
                class_num += 1
                #print("class_num: %s" %class_num)
            else:
                class_num = 0
                #print("class_num: %s" %class_num)
            if class_num >= class_thresh: 
                if broken_count >= transition_threshold or clas != origin:
                    transitions[origin + "->" + clas].append((last_whole, frame_no))
                    #print(origin + "->" + clas)
                    origin = clas
                    #print("origin: %s" %origin)
                broken_count = 0
                #print("broken_count: %s" %broken_count)
                
            
                
            last_whole = frame_no
            filtrate_len += 1 
            last_class = clas
            
            
    else:
        #print("broken")
        class_num = 0
        #print("class_num: %s" %class_num)
        broken_count += 1
        #print("broken_count: %s" %broken_count)

    return [broken_count, class_num, origin, last_class, last_whole, filtrate_len]



        
# take largest isodata image region, and return aclassification
def sideify(idir):
    iner = idir.inertia_tensor[0, 0] + idir.inertia_tensor[1, 1]
    maal = idir.major_axis_length
    mial = idir.minor_axis_length
    sol = idir.solidity 
    ecc = idir.eccentricity
    cva = idir.convex_area
    area = idir.area
    if area < 1280:
        return "ucf"
    elif ecc <= 0.7 and  mial > 41:
        return "t"
    elif ecc > 0.8 and ecc < 0.91 and maal > 66.5 and maal < 74: # maal > 67.5 and or (mial < 41 and maal > 65): # it's on top
        return "c"
    else:
        return "ucf"
    """iner = idir.inertia_tensor[0, 0] + idir.inertia_tensor[1, 1]
    maal = idir.major_axis_length
    mial = idir.minor_axis_length
    sol = idir.solidity 
    ecc = idir.eccentricity
    cva = idir.convex_area
    if sol <= 0.725 or (iner> 405 and mial > 43.5): # sol <.72
        return "bm"
    elif ecc > 0.8 or maal > 65: # or (mial < 41 and maal > 65): # it's on top #ec >.82
        return "t"
    elif ecc < 0.75 or maal < 64.5: #or mial > 42  #and cva > 2200 and iner < 325: #or (maal < 64 and iner < 360): # it's on bottom
        return "b"
    else:
        return "ucf"
        """




def coord_prod(thresh_y, thresh_i):
    m_rop = measure.regionprops
    full = largest_region_extractor(measure.label(thresh_y))
    four = largest_region_extractor(measure.label(thresh_i))
    full_or = m_rop(full.astype(int), coordinates = 'rc')[0].orientation
    one_and_some = full^four
    one = largest_region_extractor(measure.label(one_and_some))


    #Begin finding substitue for major and minor axes herre
    # use min and maax x and  values to find the geometric center
    # or use those to substitue for eigenvectors 
    rotated_one = ndimage.rotate(one, math.degrees(-full_or), reshape = False)
    rotated_full = ndimage.rotate(full, math.degrees(-full_or), reshape = False)
    geo_cent = m_rop(rotated_full.astype(int), coordinates = 'rc')[0].bbox
    #print(geo_cent)
    ave = np.average
    cent_x = ave([geo_cent[1], geo_cent[3]]) 
    cent_y = ave([geo_cent[0], geo_cent[2]])
    #print(cent_x, cent_y)

    raw_coords = (m_rop(rotated_full.astype(int), coordinates = 'rc')[0].coords).T
    ys = raw_coords[0]
    xs = raw_coords[1]
    x = xs - cent_x
    y = ys - cent_y      

    one_com = m_rop(rotated_one.astype(int))[0].centroid

    rel_com_one = (one_com[1] - cent_x, one_com[0] - cent_y)
    product = np.prod(rel_com_one)
    return product
        
def rotate_point(point, angle):
    x0 = point[0]
    y0 = point[1]
    x1 = x0*np.cos(angle) - y0*np.sin(angle)
    y1 = x0*np.sin(angle) + y0*np.cos(angle)
    return (x1, y1)
 
def region_selector(labeled_image, label):
    #print(label)
    x = labeled_image == label
    #plt.imshow(x)
    return x

# make a function that takes a set of labeled regions, and then  returns a boolean array containing only the largest
def largest_region_extractor(labeled_regions_set):
    props_lists = measure.regionprops(labeled_regions_set)
    #print(len(labeled_regions_set), len(props_lists))
    biggest_r_label = max(props_lists, key = attrgetter('area')).label
    return region_selector(labeled_regions_set, biggest_r_label)    
#from pair of labelled, thresholded images, return an image category (of three)

#from pair of labelled,thresholded images, return the x/y sign or x/y magnitude for the transparent dot


In [4]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/2ND*/4*[0-9].avi")#["./tchakamau/New_setup/45681_10Vpp_25.avi"
print(filenames)
num_top_keys = 3
num_full_keys = 3
num_tran_keys = 2
trans_threshes = [30, 40, 50]#[5, 8, 10, 19, 27, 38, 52, 60]
class_thresh = 22
trans_runs = {}
#trans_fil = 10

ave = np.average

area_threshold = 1875   # must have enough particles
iner_threshold = (340, 500)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram # minor_axis length (< filterd) # particles must be in parallelogram
#major_thresholds = (59, 76)
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [area_threshold , iner_threshold ]
params = [1280,  0.7 ,41,0.8,0.91,66.5,74]
print("Number files: ", len(filenames))

for trans_fil in trans_threshes:
    print("T threshold: %s" %trans_fil)    
    Top_bottoms = np.zeros((len(filenames), num_top_keys))
    Full_classif = np.zeros((len(filenames), num_full_keys))
    N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
    num_total_frames = 0 
    filtrate_len = 0
    for vidnum in range(len(filenames)):
        print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
        #meta = ffprobe(filenames[vidnum])
        #print(meta)#['@nb_frames'])
        #print(5)
        frames = vreader(filenames[vidnum])
        #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
        #print(frames)



        sides = {"t":[], "c":[], "ucf":[]}
        classes = {"t":[], "c":[], "ucf":[]}
        transitions = [("t->t",[]), ("t->c",[]),
                      ("c->t" ,[]), ("c->c",[])]

        transitions = OrderedDict(transitions)
        broken_count = 0
        class_num = 0
        origin = 'ucf'
        last_class = 'ucf'
        last_whole = 0
        for fr in frames:
            frame = fr[:, :, 2]
            num_total_frames += 1
            org = total_threshold_filter(frame, num_total_frames, trans_fil, class_thresh,
                                         broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                                         sides, classes, transitions)
            broken_count, class_num, origin, last_class, last_whole, filtrate_len = org
        filtered_len = filtrate_len
        #----------------------------------------------------------------
        trans_len = len([y for x in transitions.values() for y in x])
        skeys = list(sides.keys())
        ckeys = list(classes.keys())
        tkeys = list(transitions.keys())
        slcs = [len(sides[x]) for x in skeys]
        clcs = [len(classes[x]) for x in ckeys]
        trans = [len(transitions[x]) for x in tkeys] #num of each transition
        ucf = len(sides["ucf"])


        Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
        Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
        N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))
        print(N_transitions[vidnum,:])
        #print("finished vid %s" %vidnum)
    trans_runs[trans_fil] = [[Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys,
                             num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_round2.npy',trans_runs)


['./tchakamau\\2ND_4_PRTS\\45661_10Vpp_0.avi', './tchakamau\\2ND_4_PRTS\\45751_10Vpp_1.avi', './tchakamau\\2ND_4_PRTS\\45751_10Vpp_2.avi', './tchakamau\\2ND_4_PRTS\\45751_10Vpp_3.avi', './tchakamau\\2ND_4_PRTS\\45801_10Vpp_4.avi', './tchakamau\\2ND_4_PRTS\\45821_10Vpp_5.avi', './tchakamau\\2ND_4_PRTS\\45861_10Vpp_6.avi', './tchakamau\\2ND_4_PRTS\\45861_10Vpp_7.avi']
Number files:  8
T threshold: 30
Processing vid 0 : ./tchakamau\2ND_4_PRTS\45661_10Vpp_0.avi
[[12. 10.]
 [10. 15.]]
Processing vid 1 : ./tchakamau\2ND_4_PRTS\45751_10Vpp_1.avi
[[18. 33.]
 [33. 46.]]
Processing vid 2 : ./tchakamau\2ND_4_PRTS\45751_10Vpp_2.avi
[[ 5.  3.]
 [ 3. 13.]]
Processing vid 3 : ./tchakamau\2ND_4_PRTS\45751_10Vpp_3.avi
[[ 9.  8.]
 [ 9. 27.]]
Processing vid 4 : ./tchakamau\2ND_4_PRTS\45801_10Vpp_4.avi
[[16.  6.]
 [ 5.  6.]]
Processing vid 5 : ./tchakamau\2ND_4_PRTS\45821_10Vpp_5.avi
[[16.  9.]
 [10. 21.]]
Processing vid 6 : ./tchakamau\2ND_4_PRTS\45861_10Vpp_6.avi
[[33. 12.]
 [12. 52.]]
Processing vid 7 

In [14]:
"""Pipeline for lists of frames"""
""""""
"""Pipeline for videos:"""
# initialize frame lists
filenames = glob("./tchakamau/2ND*/4*[0-9].avi")#["./tchakamau/New_setup/45681_10Vpp_25.avi"
print(filenames)
num_top_keys = 3
num_full_keys = 3
num_tran_keys = 2
trans_threshes = [60, 70, 80]#[5, 8, 10, 19, 27, 38, 52, 60]
class_thresh = 22
trans_runs = {}
#trans_fil = 10

ave = np.average

area_threshold = 1875   # must have enough particles
iner_threshold = (340, 500)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram # minor_axis length (< filterd) # particles must be in parallelogram
#major_thresholds = (59, 76)
"""area_threshold = 1800   # must have enough particles
convex_thresholds = (2000, 2800)   # convex hull area (> 2500 filtered)  # particles must be in parallelogram
minor_thresholds = (38, 48) # minor_axis length (< filterd) # particles must be in parallelogram
major_thresholds = (59, 76)"""
thresholds = [area_threshold , iner_threshold ]
params = [1280,  0.7 ,41,0.8,0.91,66.5,74]
print("Number files: ", len(filenames))

for trans_fil in trans_threshes:
    print("T threshold: %s" %trans_fil)    
    Top_bottoms = np.zeros((len(filenames), num_top_keys))
    Full_classif = np.zeros((len(filenames), num_full_keys))
    N_transitions = np.zeros((len(filenames), num_tran_keys, num_tran_keys))
    num_total_frames = 0 
    filtrate_len = 0
    for vidnum in range(len(filenames)):
        print("Processing vid %s : %s" %(vidnum, filenames[vidnum]))
        #meta = ffprobe(filenames[vidnum])
        #print(meta)#['@nb_frames'])
        #print(5)
        frames = vreader(filenames[vidnum])
        #frame_vid = frames[:, :, :, 2]# making videos ino a frame list
        #print(frames)



        sides = {"t":[], "c":[], "ucf":[]}
        classes = {"t":[], "c":[], "ucf":[]}
        transitions = [("t->t",[]), ("t->c",[]),
                      ("c->t" ,[]), ("c->c",[])]

        transitions = OrderedDict(transitions)
        broken_count = 0
        class_num = 0
        origin = 'ucf'
        last_class = 'ucf'
        last_whole = 0
        for fr in frames:
            frame = fr[:, :, 2]
            num_total_frames += 1
            org = total_threshold_filter(frame, num_total_frames, trans_fil, class_thresh,
                                         broken_count, class_num, origin, last_class, last_whole, filtrate_len, 
                                         sides, classes, transitions)
            broken_count, class_num, origin, last_class, last_whole, filtrate_len = org
        filtered_len = filtrate_len
        #----------------------------------------------------------------
        trans_len = len([y for x in transitions.values() for y in x])
        skeys = list(sides.keys())
        ckeys = list(classes.keys())
        tkeys = list(transitions.keys())
        slcs = [len(sides[x]) for x in skeys]
        clcs = [len(classes[x]) for x in ckeys]
        trans = [len(transitions[x]) for x in tkeys] #num of each transition
        ucf = len(sides["ucf"])


        Top_bottoms[vidnum,:] = [slcs[i] for i in range(len(skeys))]
        Full_classif[vidnum,:] = [clcs[i] for i in range(len(ckeys))]
        N_transitions[vidnum,:] = np.reshape(trans, (num_tran_keys, num_tran_keys))
        print(N_transitions[vidnum,:])
        #print("finished vid %s" %vidnum)
    trans_runs[trans_fil] = [[Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys,
                             num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames], thresholds, params]
        #---------------------------------------------------------------------------------

#-----------------------------------------------------------------------

np.save('trans_round2_2.npy',trans_runs)


['./tchakamau\\2ND_4_PRTS\\45661_10Vpp_0.avi', './tchakamau\\2ND_4_PRTS\\45751_10Vpp_1.avi', './tchakamau\\2ND_4_PRTS\\45751_10Vpp_2.avi', './tchakamau\\2ND_4_PRTS\\45751_10Vpp_3.avi', './tchakamau\\2ND_4_PRTS\\45801_10Vpp_4.avi', './tchakamau\\2ND_4_PRTS\\45821_10Vpp_5.avi', './tchakamau\\2ND_4_PRTS\\45861_10Vpp_6.avi', './tchakamau\\2ND_4_PRTS\\45861_10Vpp_7.avi']
Number files:  8
T threshold: 60
Processing vid 0 : ./tchakamau\2ND_4_PRTS\45661_10Vpp_0.avi
[[ 8. 10.]
 [10.  8.]]
Processing vid 1 : ./tchakamau\2ND_4_PRTS\45751_10Vpp_1.avi
[[ 9. 33.]
 [33. 30.]]
Processing vid 2 : ./tchakamau\2ND_4_PRTS\45751_10Vpp_2.avi
[[2. 3.]
 [3. 9.]]
Processing vid 3 : ./tchakamau\2ND_4_PRTS\45751_10Vpp_3.avi
[[ 1.  8.]
 [ 9. 16.]]
Processing vid 4 : ./tchakamau\2ND_4_PRTS\45801_10Vpp_4.avi
[[7. 6.]
 [5. 6.]]
Processing vid 5 : ./tchakamau\2ND_4_PRTS\45821_10Vpp_5.avi
[[ 7.  9.]
 [10. 16.]]
Processing vid 6 : ./tchakamau\2ND_4_PRTS\45861_10Vpp_6.avi
[[15. 12.]
 [12. 36.]]
Processing vid 7 : ./tcha

In [17]:
trans_runs= np.load('trans_round2_2.npy')[()]
trans_threshes = [60, 70, 80]
"""num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}"""

skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    #Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys, num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil][0]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(nu   m_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    Nt_sums_broad =Nt_sums
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    """print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]"""
    #ERROR ANALSIS HERE
    print("\nErrors in transition matrix ((row_sums - column_sums) / 2)")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))/2))
    print("\nError %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))*100 /(sums(Nt_sums, axis = 1) + sums(Nt_sums, axis =0))))
    
    print("\nTransition matrix, broad")
    Nt_percs_broad = Nt_percs
    print(Nt_percs_broad)
    """print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    """
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    print("\n%s of transitions to states broad:")
    [print(sums(Nt_percs_broad, axis = 0)*100/sums(Nt_percs_broad))]
    print("\n%s of transitions from states broad:")
    [print(sums(Nt_percs_broad, axis = 1)*100/sums(Nt_percs_broad))]
    #print("\n%s of transitions narrow")


TRANSITION THRESHOLD  60

sums in broad classifications
t sum 158937
c sum 247469
ucf sum 48857
Total:  455263.0

sums in narrow classifications
t sum 158937
c sum 247469
ucf sum 48857
Total:  455263.0

sums of transitions broad
[[ 54.  87.]
 [ 87. 132.]]

sums of transitions narrow
[[ 54.  87.]
 [ 87. 132.]]
Total:  360.0

Errors in transition matrix ((row_sums - column_sums) / 2)
[0. 0.]

Error %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))
[0. 0.]

Transition matrix, broad
[[15.         24.16666667]
 [24.16666667 36.66666667]]

Transition matrix, narrow
[[15.         24.16666667]
 [24.16666667 36.66666667]]
100.0

%s of transitions to states narrow:
[39.16666667 60.83333333]

%s of transitions from states narrow:
[39.16666667 60.83333333]

%s of transitions to states broad:
[39.16666667 60.83333333]

%s of transitions from states broad:
[39.16666667 60.83333333]

TRANSITION THRESHOLD  70

sums in broad classifications
t sum 158937
c sum 247469
u

In [13]:
trans_runs= np.load('trans_round2.npy')[()]
trans_threshes = [30, 40, 50]
"""num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}"""

skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    #Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys, num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil][0]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(nu   m_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    Nt_sums_broad =Nt_sums
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    """print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]"""
    #ERROR ANALSIS HERE
    print("\nErrors in transition matrix ((row_sums - column_sums) / 2)")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))/2))
    print("\nError %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))*100 /(sums(Nt_sums, axis = 1) + sums(Nt_sums, axis =0))))
    
    print("\nTransition matrix, broad")
    Nt_percs_broad = Nt_percs
    print(Nt_percs_broad)
    """print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    """
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    print("\n%s of transitions to states broad:")
    [print(sums(Nt_percs_broad, axis = 0)*100/sums(Nt_percs_broad))]
    print("\n%s of transitions from states broad:")
    [print(sums(Nt_percs_broad, axis = 1)*100/sums(Nt_percs_broad))]
    #print("\n%s of transitions narrow")


TRANSITION THRESHOLD  30

sums in broad classifications
t sum 158937
c sum 247469
ucf sum 48857
Total:  455263.0

sums in narrow classifications
t sum 158937
c sum 247469
ucf sum 48857
Total:  455263.0

sums of transitions broad
[[116.  87.]
 [ 87. 199.]]

sums of transitions narrow
[[116.  87.]
 [ 87. 199.]]
Total:  489.0

Errors in transition matrix ((row_sums - column_sums) / 2)
[0. 0.]

Error %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))
[0. 0.]

Transition matrix, broad
[[23.72188139 17.79141104]
 [17.79141104 40.69529652]]

Transition matrix, narrow
[[23.72188139 17.79141104]
 [17.79141104 40.69529652]]
100.0

%s of transitions to states narrow:
[41.51329243 58.48670757]

%s of transitions from states narrow:
[41.51329243 58.48670757]

%s of transitions to states broad:
[41.51329243 58.48670757]

%s of transitions from states broad:
[41.51329243 58.48670757]

TRANSITION THRESHOLD  40

sums in broad classifications
t sum 158937
c sum 247469
u

In [15]:
trans_runs= np.load('trans_runs2.npy')[()]
"""num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}"""

skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    #Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys, num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(nu   m_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    T_eb = np.vstack([Nt_sums[0], 
                                 (Nt_sums[1] + Nt_sums[2]) , 
                                 (Nt_sums[3] + Nt_sums[4])])
    Nt_sums_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    """print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]"""
    #ERROR ANALSIS HERE
    print("\nErrors in transition matrix ((row_sums - column_sums) / 2)")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))/2))
    print("\nError %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))")
    print(((sums(Nt_sums, axis = 1) - sums(Nt_sums, axis =0))*100 /(sums(Nt_sums, axis = 1) + sums(Nt_sums, axis =0))))
    
    print("\nTransition matrix, broad")
    T_eb = np.vstack([Nt_percs[0], 
                                 (Nt_percs[1] + Nt_percs[2]) , 
                                 (Nt_percs[3] + Nt_percs[4])])
    Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_percs_broad)
    """print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    """
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    print("\n%s of transitions to states broad:")
    [print(sums(Nt_percs_broad, axis = 0)*100/sums(Nt_percs_broad))]
    print("\n%s of transitions from states broad:")
    [print(sums(Nt_percs_broad, axis = 1)*100/sums(Nt_percs_broad))]
    #print("\n%s of transitions narrow")


TRANSITION THRESHOLD  60

sums in broad classifications
bm sum 84370
b sum 109454
t sum 83204
ucf sum 92566
Total:  369594.0

sums in narrow classifications
bm sum 84370
br sum 73791
bl sum 35663
tr sum 40931
tl sum 42273
ucf sum 92566
Total:  369594.0

sums of transitions broad
[[ 54.  21.  70.]
 [ 15.  70.  42.]
 [ 77.  34. 262.]]

sums of transitions narrow
[[ 54.  12.   9.  25.  45.]
 [  4.   6.  13.  10.  13.]
 [ 11.  14.  37.   8.  11.]
 [ 27.   3.  13. 161.  19.]
 [ 50.  10.   8.  19.  63.]]
Total:  645.0

Errors in transition matrix ((row_sums - column_sums) / 2)
[-0.5  0.5  0.5  0.  -0.5]

Error %%s in transition matrix ((row_sums - column_sums) *100 / (row_sums + column_sums))
[-0.34364261  1.0989011   0.62111801  0.         -0.33222591]

Transition matrix, broad
[[ 8.37209302  3.25581395 10.85271318]
 [ 2.3255814  10.85271318  6.51162791]
 [11.9379845   5.27131783 40.62015504]]

Transition matrix, narrow
[[ 8.37209302  1.86046512  1.39534884  3.87596899  6.97674419]
 [ 0.62

KeyError: 70

In [ ]:
trans_runs= np.load('trans_runs1.npy')[()]
"""num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}"""

skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    #Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys, num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(num_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    T_eb = np.vstack([Nt_sums[0], 
                                 (Nt_sums[1] + Nt_sums[2]) , 
                                 (Nt_sums[3] + Nt_sums[4])])
    Nt_sums_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    """print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]"""
    #ERROR ANALSIS HERE
    print("Errors in transition matrix (row_sums - column_sums / 2)")
    print((sums(Nt_percs_broad, axis = 1) - sums(Nt_percs_broad, axis = 1) / 2))
    
    print("Transition matrix, broad")
    T_eb = np.vstack([Nt_percs[0], 
                                 (Nt_percs[1] + Nt_percs[2]) , 
                                 (Nt_percs[3] + Nt_percs[4])])
    Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_percs_broad)
    print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    #print("\n%s of transitions narrow")
    #[print(tkeys[i], " %.f%%" %Nt_percs.flatten()[i]) for i in range(len(tkeys))]

In [ ]:
trans_runs= np.load('trans_runs.npy')[()]
num_top_keys = 4
num_full_keys = 6
num_tran_keys = 5
trans_threshes = [5, 8, 10, 19, 27, 38, 52, 60]
sides = {"bm":[], "b":[], "t":[], "ucf":[]}
classes = {"bm":[], "br":[], "bl":[], "tr":[], "tl":[], "ucf":[]}
transitions = {"bm->bm" :[], "bm->br":[], "bm->bl":[], "bm->tr":[], "bm->tl":[],
              "br->bm" :[], "br->br":[], "br->bl":[], "br->tr":[], "br->tl":[],
              "bl->bm" :[], "bl->br":[], "bl->bl":[], "bl->tr":[], "bl->tl":[],
              "tr->bm" :[], "tr->br":[], "tr->bl":[], "tr->tr":[], "tr->tl":[],
              "tl->bm" :[], "tl->br":[], "tl->bl":[], "tl->tr":[], "tl->tl":[]}

skeys = list(sides.keys())
ckeys = list(classes.keys())
for trans_fil in trans_threshes:
    print("\nTRANSITION THRESHOLD ", trans_fil)
    
    Top_bottoms, Full_classif, N_transitions = trans_runs[trans_fil]
    #Top_bottoms, Full_classif, N_transitions, trans_threshes, num_top_keys, num_full_keys,
                             #num_tran_keys, skeys, ckeys, num_tran_keys, num_total_frames = trans_runs[trans_fil]
    sums = np.sum
    Tb_sums = [sums(Top_bottoms[:, c]) for c in range(num_top_keys)]
    Fc_sums = [sums(Full_classif[:, c]) for c in range(num_full_keys)]
    Nt_sums = sums(N_transitions, axis = 0)#[sums(N_transitions[:, c]) for c in range(num_tran_keys)]
    #print("\nNumber of frames obbserved: ", num_total_frames)

    Tb_percs = [Tb_sums[c] *100 /sums(Tb_sums) for c in range(num_top_keys)]
    Fc_percs = [Fc_sums[c] *100/sums(Fc_sums) for c in range(num_full_keys)]
    Nt_percs = Nt_sums *100/sums(Nt_sums)
    #print(sums(N_transitions), sums(Nt_sums))


    print("\nsums in broad classifications")
    [print(skeys[i], "sum %.f" %Tb_sums[i]) for i in range(len(skeys))]
    print("Total: ", sums(Tb_sums) )
    print("\nsums in narrow classifications")
    [print(ckeys[i], "sum %.f" %Fc_sums[i]) for i in range(len(ckeys))]
    print("Total: ", sums(Fc_sums ) )
    print("\nsums of transitions broad")
    T_eb = np.vstack([Nt_sums[0], 
                                 (Nt_sums[1] + Nt_sums[2]) , 
                                 (Nt_sums[3] + Nt_sums[4])])
    Nt_sums_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_sums_broad)
    print("\nsums of transitions narrow")
    print(Nt_sums)
    print("Total: ", sums(Nt_sums) )



    print("\n%s in broad classifications")
    [print(skeys[i], " %.f%%" %Tb_percs[i]) for i in range(len(skeys))]
    print("\n%s in narrow classifications")
    [print(ckeys[i], " %.f%%" %Fc_percs[i]) for i in range(len(ckeys))]
    print("Transition matrix, broad")
    T_eb = np.vstack([Nt_percs[0], 
                                 (Nt_percs[1] + Nt_percs[2]) , 
                                 (Nt_percs[3] + Nt_percs[4])])
    Nt_percs_broad = np.column_stack([T_eb[:, 0], 
                                 (T_eb[:, 1]+ T_eb[:, 2]), 
                                 (T_eb[:, 3]+ T_eb[:, 4])])
    print(Nt_percs_broad)
    print("row_sums of transitin matrix, broad")
    print(sums(Nt_percs_broad, axis = 1))
    print("\n%s of transitions broad(->bm, ->b, ->t):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 0)[0]*100/sums(Nt_sums), 
           (sums(Nt_sums, axis = 0)[1] + sums(Nt_sums, axis = 0)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 0)[3] + sums(Nt_sums, axis = 0)[4])*100/(sums(Nt_sums))])
    print("\n%s of transitions broad(bm->, b->, t->):")
    #print("->bm %.f%%" %(Nt_percs[0, 0] + sums(Nt_percs[1:3, 0]) + sums(Nt_percs[3:5, 0])))
    print([sums(Nt_sums, axis = 1)[0]*100/sums(Nt_sums),
           (sums(Nt_sums, axis = 1)[1] + sums(Nt_sums, axis = 1)[2])*100/(sums(Nt_sums)),
           (sums(Nt_sums, axis = 1)[3] + sums(Nt_sums, axis =1)[4])*100/(sums(Nt_sums))])#(Nt_percs[i,1] + Nt_percs[i,1+1] + Nt_percs[i+1, 1] + Nt_percs[i+1, 1+1] + sums(Nt_percs[0, 1:3]))) for i in [1,3]]
    #print("->t %.f%%" %(Nt_percs[i,2] + Nt_percs[i,2+1] + Nt_percs[i+1, 2] + Nt_percs[i+1, 2+1] + sums(Nt_percs[0, 3:5]))) for i in [1,3]]
    print("\nTransition matrix, narrow")
    print(Nt_percs)
    print(sums(Nt_percs))
    print("\n%s of transitions to states narrow:")
    [print(sums(Nt_percs, axis = 0)*100/sums(Nt_percs))]
    print("\n%s of transitions from states narrow:")
    [print(sums(Nt_percs, axis = 1)*100/sums(Nt_percs))]
    #print("\n%s of transitions narrow")
    #[print(tkeys[i], " %.f%%" %Nt_percs.flatten()[i]) for i in range(len(tkeys))]